<a href="https://colab.research.google.com/github/tomkart/musicgen-colab/blob/main/musicgen_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Fri Feb  7 07:19:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
| 31%   40C    P2             62W /  450W |     502MiB /  24564MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu128


In [4]:
!pip install transformers datasets

In [5]:
import torch
from transformers import MusicgenForConditionalGeneration, MusicgenProcessor, pipeline
import scipy
from IPython.display import Audio
import numpy as np

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [7]:
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-large").to(device)
processor = MusicgenProcessor.from_pretrained("facebook/musicgen-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5Encoder

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
text_prompt = "90s rock song with loud guitars with bassy drumss"
# The processor expects a dictionary with "text" key

# Generate the input IDs and attention mask from the text prompt.
inputs = processor(text=text_prompt, padding=True,return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = inputs.to("cuda")

audio_values = model.generate(**inputs,  max_new_tokens=1024)

`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


In [9]:
# Convert the output to a numpy array and play it
audio = audio_values[0].cpu().numpy()


sampling_rate = model.config.audio_encoder.sampling_rate
# Move the tensor to CPU before converting to NumPy array
scipy.io.wavfile.write("musicgen_large.wav", rate=sampling_rate, data=audio_values[0, 0].cpu().numpy())
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate) # Move tensor to cpu before using numpy()